In [1]:
### import pandas as pd
import numpy as np
import glob
import json
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '../')
# from utilities import *
from sklearn.metrics.pairwise import cosine_similarity
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!pip install nltk

In [3]:
# import nltk
# nltk.download("all")

In [4]:
!pip install transformers

In [5]:
from nltk import tokenize
import nltk
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F

In [6]:
def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents


In [7]:
import pandas as pd
combined_df  = pd.read_csv(r"/kaggle/input/training-data-clean/my_data.csv")

In [8]:
combined_df.head()

,Unnamed: 0,ctext,text
0,0,Registered office C - Emcee 44 Southampton B...,08 Mediterranean Oil & Gas Plc Annual Report ...
1,1,Mediterranean Oil & Gas Plc Annual Report &...,Mediterranean Oil & Gas Plc Annual Report &...
2,2,Mediterranean Oil & Gas Plc Annual Report &...,Mediterranean Oil & Gas Plc Annual Report &...
3,3,ANNUAL REPORT AND ACCOUNTS 2013 Download the...,4 MEGGITT PLC REPORT AND ACCOUNTS 2013 Chief ...
4,4,MEGGITT PLC ANNUAL REPORT AND ACCOUNTS | 20...,6 MEGGITT PLC REPORT AND ACCOUNTS ...


In [9]:
combined_df = combined_df.drop(columns=['Unnamed: 0'])


In [10]:
combined_df.head()

,ctext,text
0,Registered office C - Emcee 44 Southampton B...,08 Mediterranean Oil & Gas Plc Annual Report ...
1,Mediterranean Oil & Gas Plc Annual Report &...,Mediterranean Oil & Gas Plc Annual Report &...
2,Mediterranean Oil & Gas Plc Annual Report &...,Mediterranean Oil & Gas Plc Annual Report &...
3,ANNUAL REPORT AND ACCOUNTS 2013 Download the...,4 MEGGITT PLC REPORT AND ACCOUNTS 2013 Chief ...
4,MEGGITT PLC ANNUAL REPORT AND ACCOUNTS | 20...,6 MEGGITT PLC REPORT AND ACCOUNTS ...


In [11]:
combined_df["ctext"][373] , combined_df["text"][373] 

(' ',
 " General Review I am pleased to report that during 2007 the Abacus Group grew its sales 12% and its operating profit, before amortisation and exceptionals, by 13% (59% higher, after amortisation  and exceptionals) when compared to 2006. The integration  of our acquisitions made in 2006 was completed ahead  of our original schedule and this was a significant contributor to the result. However, although the year started as detailed in our 2006 review with a positive Book to Bill and Billings in line with management expectations, as reported on 31 January 2007  at our AGM, this trend changed quickly as often happens in the electronic component industry. The UK and Italy were the first countries to see this change and although Group Billings for the first half remained in line with our expectations, the Bookings did not, a fact we brought to the markets attention with our statement of 30 March 2007. This lower Booking trend continued as we moved into the second half of 2007 and, wh

In [12]:
from transformers import AutoTokenizer
from transformers import  BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert_model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = False).to("cuda")


In [13]:
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [14]:
def get_sen_encoding(sents):
    '''
    Function to generate encoding for each word in the input list 
    input: sents - List of sentences
    returns the list of the sentence encoding 
    '''
    max_seq_length = 150
    a = 0.001
    answer = None
    for sent in sents:
        ip =tokenizer(sent, return_tensors='pt',truncation=True,max_length=150, padding="max_length")
        tokens = tokenizer.convert_ids_to_tokens(ip['input_ids'][0])
        ip = ip.to("cuda")
        bert_output = bert_model(**ip)
        embedding = bert_output['pooler_output'].clone().detach()
        embedding = embedding.to("cpu")
        if answer == None:
            answer = embedding
            answer.resize_(1, 768)
        else:
            embedding.resize_(1, 768)
            answer = torch.cat((answer, embedding),0)
    return answer

In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def similarity_l_l(l1, l2):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary 
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary 
    '''
    l = l1 + l2
    sents_encodings = get_sen_encoding(l)
    similarities = cosine_similarity(sents_encodings)

    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        
        if len(vals) == 0:
            result.append(-1)  # Handle the case where vals is empty as needed
        else:
            idx = np.argmax(vals)
            result.append(idx)
    
    return result


In [16]:
def get_chunks_data_from_docV2(doc, summ):
    '''
    Function to generate chunks along with their summaries 
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries 
    '''
    chunk_summ_word_threshold = 150
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)
    
    result = (similarity_l_l(summ_sents,doc_sents))
    
    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]
    
    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 512):
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:
                summ = summ + sentence_mapping[chunk_sent]
        if len(tokenizer.tokenize(summ)) >= chunk_summ_word_threshold:
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ

In [17]:
combined_df = combined_df[2500:]

In [18]:
data_source = combined_df["ctext"] 

In [19]:
data_source .shape

(500,)

In [20]:
data_summary = combined_df["text"] 

In [21]:
combined_df

,ctext,text
2500,"1 Contents Page Directors, Secretary and Adv...",5 I am delighted to present to shareholders m...
2501,"9 1 Contents Page Directors, Secretary and Ad...",5 I am pleased to report another encouraging ...
2502,ANNUAL REPORT & ACCOUNTS2003 ANNUAL REPORT & ...,03 CHIEF EXECUTIVES REVIEW Overview After a s...
2503,enabling optical technologies ++++++++++++ ++...,Performance For the year ended 30 September 2...
2504,ANNUAL REPORT + ACCOUNTS 2012 www.goochandhou...,GOOCH & HOUSEGO PLC ANNUAL REPORT 2012 8 CHIE...
...,...,...
2995,Mears Group PLC Annual report and accounts...,06 Mears Group PLC Annual report and acco...
2996,Making a positive difference... Mears Group...,08 Mears Group PLC Annual report and acco...
2997,Annual report and accounts 2008 Making conten...,14 Mecom Group plc Annual report and accounts...
2998,report and financial statements 2004 UK Medi...,chief executives report The year ended 30 Sep...


In [22]:
# def get_root_path():
#     '''
#     function to get root path of dataset

#     change the path variable to the path of the dataset
#     '''
#     path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
#     return path

# def get_summary_data(dataset, train):
#     '''
#     function to get names, documents, and summaries

#     change the path variable to the path of the dataset
#     '''
#     if dataset == "N2":
#         path = get_root_path() + '/N2/Full-Text/India'
#         all_files = glob.glob(path + "/*.txt")

#         data_source = []
#         names = []
#         for filename in all_files:
#             with open(filename, 'r') as f: 
#                 p = filename.rfind("/")
# #                 print(filename[p+1:])
#                 names.append(filename[p+1:])
#                 a = f.read()
#                 data_source.append(a)/
#         return names, data_source, []
    
#     path = get_root_path() + '/Summary-Data-' + dataset + '/' + train + '-data/judgement'
#     all_files = glob.glob(path + "/*.txt")
#     data_source = []
#     names = []
#     for filename in all_files:
#         with open(filename, 'r') as f:
#             p = filename.rfind("/")
#             names.append(filename[p+1:])
#             a = f.read()
#             data_source.append(a)
#     path = get_root_path() + '/Summary-Data-' + dataset + '/' + train + '-data/summary'
#     all_files = glob.glob(path + "/*.txt")
#     data_summary = []
#     for filename in all_files:
#         with open(filename, 'r') as f: 
#             a = f.read()
#             l = len(a)
#             data_summary.append(a)
            
#     return names, data_source, data_summary

# def get_summary_data_rhet_train(dataset):
#     '''
#     function to get names, documents, and summaries for Rhetorical labeled documents - train

#     change the path variable to the path of the Rhetorical labeled dataset
#     '''
#     path = get_root_path() + '/rhet/' + dataset.lower() + '_ft_rhet' # use your path
#     all_files = glob.glob(path + "/*.txt")

#     data_source = []
#     names = []
#     for filename in all_files:
#         with open(filename, 'r') as f: 
#             p = filename.rfind("/")
#             names.append(filename[p+1:])
#             a = f.read()
#             data_source.append(a)

#     path = get_root_path() + '/rhet/RhetSumm_Dataset/raw_files/'+ dataset +'/summary' # use your path
#     all_files = glob.glob(path + "/*.txt")

#     data_summary = {}
#     for filename in all_files:
#         with open(filename, 'r') as f: 
#             p = filename.rfind("/")
#             a = f.read()
#             l = len(a)
#             data_summary[filename[p+1:]] = (a)
#     return names, data_source, data_summary

# def get_summary_data_rhet_test(dataset):
#     '''
#     function to get names, documents, and summaries for Rhetorical labeled documents - test

#     change the path variable to the path of the Rhetorical labeled dataset
#     '''
#     path = get_root_path() + '/rhet/RhetSumm_Dataset/rhet/' + dataset + "/" # use your path
#     all_files = glob.glob(path + "/*.txt")

#     data_source = []
#     names = []
#     for filename in all_files:
#         with open(filename, 'r') as f: 
#             p = filename.rfind("/")
# #             print(filename[p+1:])
#             names.append(filename[p+1:])
#             a = f.read()
#             data_source.append(a)

#     return names, data_source


def get_req_len_dict(dataset, istrain):
    '''
    function to required length data for each summary

    change the path variable to the path to Summary_Length_India.txt/ Summary_Length_Uk.txt file
    '''
    
    if dataset == "N2":
        f = open(get_root_path() + "/N2/Summary_Length_India.txt", "r")
        a = (f.read())
        a = a.split("\n")
        dict_names = {}
        for i in a:
            b = i.split("	")
            dict_names[b[0] + ".txt"] = int(b[1])
        return dict_names 
    
    f = open(get_root_path() + "/Summary-Data-"+ dataset +"/length-file-" + istrain +".txt", "r")
    a = (f.read())
    a = a.split("\n")
    dict_names = {}
    for i in a:
        b = i.split("	")
        try:
            tp = int(b[2])
            dict_names[b[0]] = tp
        except:
            print(b)
    return dict_names  

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents

# def nest_sentencesV2(document,chunk_length):
#     '''
#     function to chunk a document
#     input:  document           - Input document
#             chunk_length        - chunk length
#     output: list of chunks. Each chunk is a list of sentences.
#     '''
#     nested = []
#     sent = []
#     length = 0
#     for sentence in nltk.sent_tokenize(document):
#         length += len(sentence.split(" "))
#         if length < chunk_length:
#             sent.append(sentence)
#         else:
#             nested.append(sent)
#             sent = []
#             sent.append(sentence)
#             length = 0
#     if len(sent)>0:
#         nested.append(sent)
#     return nested

def nest_sentencesV2(document, max_chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            max_chunk_length   - Maximum length for each chunk (e.g., 512 for BERT)
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        sentence_length = len(tokenizer.tokenize(sentence))
        if length + sentence_length <= max_chunk_length:
            sent.append(sentence)
            length += sentence_length
        else:
            nested.append(sent)
            sent = [sentence]  # Start a new chunk with the current sentence
            length = sentence_length

    if sent:
        nested.append(sent)

    return nested


def nest_sentencesMV2(document_sents,chunk_length):
    '''
    function to chunk a document
    input:  doc_sents           - Input document sentences
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    #modeified v2
    
    for sentence in document_sents:
        length += len((sentence.split(" ")))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

def nest_sentences(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a string.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested
  

def nest_sentencesV3(doc_sents,chunk_length, dict_sents_labels):
    '''
    function to first segment the document using rhetorical roles and then chunk if required
    input:  doc_sents           - Input document sentences
            chunk_length        - chunk length
            dict_sents_labels   - dictionary for every sentence and label
    output: list of chunks
    '''
    s = list(set(dict_sents_labels.values()))
#     print(s)
    all_chunks = []
    
    for label in s:
        doc_sents_withlabels = []
        for sent in doc_sents:
            if sent == '':continue
            if dict_sents_labels[sent] == label:
                doc_sents_withlabels.append(sent)
        chunks = nest_sentencesMV2(doc_sents_withlabels, chunk_length)
        
        edited_chunks = []
        for chunk in chunks:
            edited_chunks.append(["<" + label + ">"] + chunk)
        
        all_chunks = all_chunks + edited_chunks

    return all_chunks   

def get_doc_sens_and_labels(doc):
    '''
    Function to read a Rhetorically labeled document.
    returns a list of sentences, the label of each sentence, dictionary: keys-sentences, values-labels
    '''
    sents = []
    labels = []
    dict_sents_labels = {}
    ss = doc.split("\n")
    for i in ss:
        try:
            spt = i.split("\t")
            sents.append(spt[0])
            labels.append(spt[1])
            dict_sents_labels[spt[0]] = spt[1] 
        except:
            pass
    return sents, labels, dict_sents_labels

In [23]:
import pandas as pd
training_chunks = []
training_summs = []
for i in tqdm(range(len(data_source))):
    if len(data_source.values[i]) < 200 or len(data_summary.values[i]) < 200:
        continue
    cks, summs = get_chunks_data_from_docV2(data_source.values[i], data_summary.values[i])
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
    if i % 50 == 0:
        full = list(zip(training_chunks, training_summs))
        df = pd.DataFrame(full, columns=['Input', 'Output'])
        file_path = f"cls_{i}.csv"
        df.to_csv(file_path, index=False)  # Use to_csv to save as CSV file

full = list(zip(training_chunks, training_summs))
df = pd.DataFrame(full, columns=['Input', 'Output'])
df.to_csv("cls_all.csv", index=False)  # Use to_csv to save as CSV file


100%|██████████| 500/500 [2:21:11<00:00, 16.94s/it]


In [24]:
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [25]:
bert_model.get_adapter_state_dict

<bound method PeftAdapterMixin.get_adapter_state_dict of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [26]:
bert_model.save_pretrained("model")

In [27]:
tokenizer.save_pretrained("model")

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')